## Time-dependent boundary layer model

In [3]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import gsw
from scipy.interpolate import interp1d
from scipy.integrate import solve_ivp
import datapath

In [4]:
ds = xr.open_dataset(datapath.adcpnc())

In [5]:
lat = 35+8.4585/60  # latitude (for f calculation)
bini = 15 # index of top bin

Smooth ADCP time series
* 30-min running median to remove spikes
* 60-min running mean to smooth

In [6]:
ds['binheight'][bini]

<xarray.DataArray 'binheight' ()>
array(17.14)

In [7]:
ds['Eas_sm'] = ds['Eas'].rolling(time=6,center=True).median().rolling(time=12,center=True).mean()
ds['Nor_sm'] = ds['Nor'].rolling(time=6,center=True).median().rolling(time=12,center=True).mean()

### Calculate time derivative of ADCP velocity for model forcing

In [9]:
# add instrument height (18 inches)
zf = ds['binheight'][bini]+ 0.4572 # upper boundary height
zobs = np.array(ds['binheight'][0:(bini+1)]) + 0.4572

In [10]:
i = 1j
kappa = 0.41 # Von Karman constant
f = gsw.f(lat)

In [11]:
gi = np.isfinite(ds['Eas_sm'][:,0])
tobs = np.array((ds['time'][gi] - ds['time'][gi][0])/np.timedelta64(1,'s')) # array of times in seconds
deltat = tobs[1]-tobs[0]

In [12]:
wobs = np.array(ds['Eas_sm'][gi,0:(bini+1)]+i*ds['Nor_sm'][gi,0:(bini+1)])
dwobsdt = np.gradient(wobs,deltat,axis=0)

### Define model equations

In [13]:
def dwdt_bbl(t, w_in):
    
    global zo
    
    N = int(len(w_in)/2)
    w = w_in[:N]+i*w_in[N:]
    
    winf = np.interp(t,tobs,wobs[:,bini])
    dwinfdt = np.interp(t,tobs,dwobsdt[:,bini])

    F = dwinfdt + i*f*winf
    ustar = kappa*zo*np.abs(w[0])/dz[0]
    coef1 = 2*kappa*ustar/(dz[:-1]+dz[1:])

    wall = np.concatenate([np.atleast_1d(0+i*0),w,np.atleast_1d(winf)])
    
    dwalldt = np.nan*np.ones(np.shape(wall))+i*np.nan*np.ones(np.shape(wall))
    dwalldt[1:-1] = (F +
                     coef1*(z[1:-1]/dz[1:])*wall[2:] +
                     (-i*f - coef1*(z[1:-1]/dz[1:]+z[:-2]/dz[:-1]))*wall[1:-1] +
                     coef1*(z[:-2]/dz[:-1])*wall[:-2])
    
    dwdt = dwalldt[1:-1]
    
    dwdt_out = np.concatenate([np.real(dwdt),np.imag(dwdt)])
    
    return dwdt_out

### Run model for varying $z_o$ values

In [16]:
nzo = 7
zo_array = np.logspace(-4,-1,nzo) # roughness length array

tf = 31*86400
ntimes = len(tobs[tobs<=tf])

nz = 201

w_sol_all = np.nan*np.ones([ntimes,nz-2,nzo]) + i*np.nan*np.ones([ntimes,nz-2,nzo])
w_sol_obs_all = np.nan*np.ones([ntimes,len(zobs),nzo]) + i*np.nan*np.ones([ntimes,len(zobs),nzo])

for zi,zo in enumerate(zo_array):
    print('zo:',zo,'(',zi+1,'/',nzo,')')

    z = np.logspace(np.log10(zo),np.log10(zf),nz)
    dz = np.diff(z)
    
    w0_obs = np.interp(z,zobs[1:bini],wobs[0,1:bini])[1:-1]
    w0 = np.concatenate([np.real(w0_obs),
                           np.imag(w0_obs)])

    tic = np.datetime64('now')
    a = solve_ivp(dwdt_bbl, t_span=[tobs[0],tf], y0 = w0, t_eval = tobs[tobs<=tf], method='BDF')
    toc = np.datetime64('now')
    print(toc-tic)
    
    N = int(np.shape(a.y)[0]/2)
    t_sol = a.t
    w_sol = a.y[:N,:]+i*a.y[N:,:]
    
    fint= interp1d(z[1:-1],w_sol,axis=0,
                   bounds_error=False,fill_value = np.nan+i*np.nan)
    w_sol_obs = fint(zobs)
    
    w_sol_all[:,:,zi] = w_sol.T
    w_sol_obs_all[:,:,zi] = w_sol_obs.T

zo: 0.0001 ( 1 / 7 )
944 seconds
zo: 0.00031622776601683794 ( 2 / 7 )
734 seconds
zo: 0.001 ( 3 / 7 )
413 seconds
zo: 0.0031622776601683794 ( 4 / 7 )
449 seconds
zo: 0.01 ( 5 / 7 )
434 seconds
zo: 0.03162277660168379 ( 6 / 7 )
479 seconds
zo: 0.1 ( 7 / 7 )
516 seconds


### Save model runs

In [ ]:
bbl_sols = xr.Dataset(
        data_vars={'uobs_subset':    (('tobs', 'zobs'), np.real(wobs[tobs<=tf,:])),
                   'vobs_subset':    (('tobs', 'zobs'), np.imag(wobs[tobs<=tf,:])),
                   'usol_all': (('tobs','zsol','zo'), np.real(w_sol_all)),
                   'vsol_all': (('tobs','zsol','zo'), np.imag(w_sol_all)),
                   'usol_obs_all': (('tobs','zobs','zo'), np.real(w_sol_obs_all)),
                   'vsol_obs_all': (('tobs','zobs','zo'), np.imag(w_sol_obs_all)),                  },
        coords={'zobs': zobs,
                'tobs': tobs[tobs<=tf],
                'zo':zo_array})
bbl_sols.to_netcdf('data/bbl_model_solutions.nc',mode='w')
bbl_sols.close()